In [1]:



from decimal import Decimal # for filenames

import pdb 

import os
import sys
import tensorflow as tf
from mat4py import loadmat
import numpy as np
import pandas as pd
from Utilities.get_image_data import load_data
from Utilities.form_train_val_test_batches import form_train_val_test_batches
from Utilities.NN_FC_layerwise import FCLayerwise

from Utilities.create_data import create_new
from Utilities.L2_error_computation import error_L2
from Utilities.loss_and_accuracies import data_loss_classification, data_loss_regression
from Utilities.model_constraint import compute_interior_loss
from Utilities.optimize_layerwise import optimize
from Utilities.Net import Final_Network

from joblib import Parallel, delayed
import multiprocessing
import random





###############################################################################
#                       HyperParameters and RunOptions                        #
###############################################################################
class Hyperparameters:
    max_hidden_layers = 7# 
    num_hidden_nodes  =100
    activation        = 'relu'
    classification_act= 'linear'
    model_constraint  =  35
    regularization    = 0.0
    node_TOL          = 1e-4
    error_TOL         = 1e-4
    batch_size        = 500
    num_epochs        = 6000
    
  
    




class RunOptions:
    def __init__(self):    
        #=== Choose Which GPU to Use ===#
        self.which_gpu = '1'
        
        #=== Use L_1 Regularization ===#
        self.use_L1 = 1
        
        #=== Choose Data Set ===#
        self.data_MNIST = 0
        self.data_CIFAR10 = 0 
        self.data_CIFAR100 = 0
        self.data_regression=1
        
        #=== Random Seed ===#
        self.random_seed = 1234




###############################################################################
#                                 File Paths                                  #
###############################################################################         
class FilePaths():    
    def __init__(self, hyperp, run_options):  
        #=== Declaring File Name Components ===# 
        self.NN_type = 'FC'
        if run_options.data_MNIST == 1:
            self.dataset = 'MNIST'
        if run_options.data_CIFAR10 == 1:
            self.dataset = 'CIFAR10'
        if run_options.data_CIFAR100 == 1:
            self.dataset = 'CIFAR100'
        if run_options.data_regression == 1:
            self.dataset = 'Abalone'
        if hyperp.regularization >= 1:
            hyperp.regularization = int(hyperp.regularization)
            regularization_string = str(hyperp.regularization)
        else:
            regularization_string = str(hyperp.regularization)
            regularization_string = 'pt' + regularization_string[2:]                        
        node_TOL_string = str('%.2e' %Decimal(hyperp.node_TOL))
        node_TOL_string = node_TOL_string[-1]
        error_TOL_string = str('%.2e' %Decimal(hyperp.error_TOL))
        error_TOL_string = error_TOL_string[-1]
        
        #=== File Name ===#
        if run_options.use_L1 == 0:
            self.filename = self.dataset + '_' + self.NN_type + '_mhl%d_hl%d_eTOL%s_b%d_e%d' %(hyperp.max_hidden_layers, hyperp.num_hidden_nodes, error_TOL_string, hyperp.batch_size, hyperp.num_epochs)
        else:
            self.filename = self.dataset + '_' + self.NN_type + '_L1_mhl%d_hl%d_r%s_nTOL%s_eTOL%s_b%d_e%d' %(hyperp.max_hidden_layers, hyperp.num_hidden_nodes, regularization_string, node_TOL_string, error_TOL_string, hyperp.batch_size, hyperp.num_epochs)

        #=== Saving Trained Neural Network and Tensorboard ===#
        #self.NN_savefile_directory = 'C:/Users/Chandradut/Desktop/Sparse training/Trained_NNs/' + self.filename # Since we need to save four different types of files to save a neural network model, we need to create a new folder for each model
        self.NN_savefile_directory =  self.filename
        self.NN_savefile_name = self.NN_savefile_directory + '/' + self.filename # The file path and name for the four files
        #self.tensorboard_directory = 'C:/Users/Chandradut/Desktop/Sparse training/Tensorboard/' + self.filename

###############################################################################


            

        
        
        
   
        
        
        
        
        

In [2]:

if __name__ == "__main__":     

    trainable=2
    
    #=== Hyperparameters and Run Options ===#    
    hyperp = Hyperparameters()
  
    run_options = RunOptions()
    

    #=== File Names ===#
    file_paths = FilePaths(hyperp, run_options)
    
    #=== Load Data ===#       
    data_train, labels_train,    data_test, labels_test,    data_input_shape, num_channels, label_dimensions    = load_data(file_paths.NN_type, file_paths.dataset, run_options.random_seed) 
    
    gauss_solution=np.loadtxt("gauss_solution.data")
    gauss_solution = tf.cast(gauss_solution,tf.float32)
    

    
    gauss_points_new = loadmat('gauss_points_new.mat')
    gauss_points_new=np.array(list(gauss_points_new.values()))[-1]
    gauss_points_new = tf.cast(gauss_points_new,tf.float32)
    

    
    gauss_weights_new = loadmat('gauss_weights_new.mat')
    gauss_weights_new=np.array(list(gauss_weights_new.values()))[-1].squeeze(-1)

    
    Coordinates = loadmat('Coordinates.mat')
    Coordinates=np.array(list(Coordinates.values()))[-1]
    Coordinates = tf.cast(Coordinates,tf.float32)
    
    
    Stiffness = loadmat('Stiffness.mat')
    Stiffness=np.array(list(Stiffness.values()))[-1]
    Stiffness = tf.cast(Stiffness,tf.float32)
    
    load = loadmat('observation.mat')
    load=np.array(list(load.values()))[-1]
    load=tf.cast(load,tf.float32)
    load=tf.reshape(load,(len(load),1))
    
    Solution = loadmat('Solution.mat')
    Solution=np.array(list(Solution.values()))[-1]
    Solution=tf.cast(Solution,tf.float32)
    Solution=tf.reshape(Solution,(len(Solution),1))
    

    

    for i in range(1,2):
    #=== Initiate training ===#


    
            
        if trainable==2:
        
        
        
            #=== GPU Settings ===#
            os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
            os.environ["CUDA_VISIBLE_DEVICES"] = run_options.which_gpu
    
            #=== Neural Network ===#
            if run_options.use_L1 == 0:
                kernel_regularizer = None
                bias_regularizer = None  
            else:
                kernel_regularizer = tf.keras.regularizers.l1(hyperp.regularization)
                bias_regularizer = tf.keras.regularizers.l1(hyperp.regularization)
            
            
          

            data_train,new_label,labels_train,load,Solution=create_new(data_train, labels_train,hyperp, run_options, data_input_shape, label_dimensions,i,load,Stiffness,Coordinates,Solution)
        
            
        #=== Construct Validation Set and Batches ===# 
            data_and_labels_train, data_and_labels_val, data_and_labels_test, num_data_train, num_data_val, num_data_test,            num_batches_train, num_batches_val, num_batches_test,data_and_labels_train_new            = form_train_val_test_batches(data_train, labels_train,                                       data_test, labels_test,                                       hyperp.batch_size, new_label, run_options.random_seed)
        
        
        
        NN = FCLayerwise(hyperp, run_options, data_input_shape, label_dimensions,kernel_regularizer, bias_regularizer)    
        NN._set_inputs( data_train)
       

        

        hyperp_n=hyperp
        optimize(hyperp,hyperp_n, run_options, file_paths, NN, data_loss_classification, data_loss_regression, data_and_labels_train, data_and_labels_val, data_and_labels_test, label_dimensions, num_batches_train,data_and_labels_train_new,hyperp.batch_size,run_options.random_seed,num_data_train,i,data_input_shape,data_train,labels_train,trainable,compute_interior_loss,error_L2,gauss_solution,gauss_points_new,gauss_weights_new,Coordinates, Stiffness, load,Solution)   
       
        
        
        if not os.path.exists("WEIGHTS"):
            os.makedirs("WEIGHTS")
        NN.save_weights("WEIGHTS"+'/'+"model_weights"+str(1)+str(trainable))
        
        
        

2022-06-15 01:32:55.276320: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-15 01:32:55.303837: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2593550000 Hz
2022-06-15 01:32:55.305253: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x555bd7a89a20 executing computations on platform Host. Devices:
2022-06-15 01:32:55.305279: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
2022-06-15 01:32:55.315688: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


ListWrapper([2, 100, 100, 1])
Beginning Training
            Epoch 0            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Model: "fc_layerwise"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
W2 (Dense)                   multiple                  10100     
_________________________________________________________________
W3 (Dense)                   multiple                  10100     
_________________________________________________________________
W4 (Dense)                   multiple                  10100     
_________________________________________________________________
W5 (Dense)                   multiple                  10100     
_________________________________________________________________
W6 (Dense)                   multiple                  10100     
____________________________________________

            Epoch 28            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.13
            Epoch 29            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.13
            Epoch 30            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.13
            Epoch 31            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.13
            Epoch 32            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
            Epoch 33            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable

            Epoch 62            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.17
            Epoch 63            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
            Epoch 64            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
            Epoch 65            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
            Epoch 66            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
            Epoch 67            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable

            Epoch 96            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
            Epoch 97            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
            Epoch 98            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.15
            Epoch 99            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.17
            Epoch 100            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.17
            Epoch 101            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainab

...

            Epoch 5869            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
L2 error: 1.370e-04
            Epoch 5870            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
            Epoch 5871            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
L2 error: 1.378e-04
            Epoch 5872            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
            Epoch 5873            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
L2 error: 1.362e-04
            Epoch 5874   

Time per Batch: 0.16
            Epoch 5901            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
L2 error: 1.391e-04
            Epoch 5902            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
            Epoch 5903            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.14
L2 error: 1.409e-04
            Epoch 5904            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.13
            Epoch 5905            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.13
L2 error: 1.392e-04
    

Time per Batch: 0.16
L2 error: 2.754e-04
            Epoch 5933            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
L2 error: 1.545e-04
            Epoch 5934            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
L2 error: 2.798e-04
            Epoch 5935            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.13
L2 error: 1.555e-04
            Epoch 5936            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.13
L2 error: 2.715e-04
            Epoch 5937            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batche

Time per Batch: 0.16
L2 error: 1.452e-04
            Epoch 5964            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
L2 error: 2.737e-04
            Epoch 5965            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
L2 error: 1.431e-04
            Epoch 5966            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.13
L2 error: 2.741e-04
            Epoch 5967            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.13
L2 error: 1.384e-04
            Epoch 5968            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batche

Time per Batch: 0.16
L2 error: 1.478e-04
            Epoch 5996            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
L2 error: 2.695e-04
            Epoch 5997            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.15
L2 error: 1.465e-04
            Epoch 5998            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
L2 error: 2.831e-04
            Epoch 5999            
Abalone_FC_L1_mhl7_hl100_rpt0_nTOL4_eTOL4_b500_e6000
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 8 batches of size 500:
Time per Batch: 0.16
L2 error: 1.521e-04
     Extending Architecture     
L2 error: 1.521e-04


In [3]:
# Evaluating the stored model

hyperp = Hyperparameters()
  
run_options = RunOptions()

file_paths = FilePaths(hyperp, run_options)

#=== Load Boundary Data ===#       
data_train, labels_train,    data_test, labels_test,    data_input_shape, num_channels, label_dimensions    = load_data(file_paths.NN_type, file_paths.dataset, run_options.random_seed) 
    

Network=Final_Network(hyperp,run_options, data_input_shape, label_dimensions) 

Network.load_weights("WEIGHTS"+'/'+"model_weights"+str(1)+str(2)).expect_partial()





# Loading Gauss quadrature data for interior domain

    
gauss_weights_new = loadmat('gauss_weights_new.mat')
gauss_weights_new=np.array(list(gauss_weights_new.values()))[-1].squeeze(-1)

gauss_solution=np.loadtxt("gauss_solution.data")
gauss_solution = tf.cast(gauss_solution,tf.float32)
    

    
gauss_points_new = loadmat('gauss_points_new.mat')
gauss_points_new=np.array(list(gauss_points_new.values()))[-1]
gauss_points_new = tf.cast(gauss_points_new,tf.float32)
    

L2_error=error_L2(gauss_points_new, gauss_solution, Network,label_dimensions,hyperp,data_input_shape,1,run_options,gauss_weights_new)

print('L2 error: %.3e' %(L2_error))

L2 error: 1.521e-04
